In [1]:
import numpy as np
import cmath
import pandas as pd

In [2]:
def phi_func(g, a, w, x0):
    i = 1.0j
    mu = np.sqrt(-g + i*w)
    phi = -(2.0*mu*np.cosh(mu*x0))/(np.sinh(mu) + mu*np.cosh(mu) - a*x0*np.sinh(mu*x0))
    return phi.real

In [3]:
def G(y, a, mu, x0):
    return (a*np.cosh(y*x0)-y*np.sinh(y))/(y*y-mu*mu)

In [4]:
def d_func(mu, kappa, nu, a, x_0):
    mu_conj = np.conj(mu)
    B = 3.0*mu*(G(kappa, a, mu, x_0) + G(nu, a, mu, x_0) + 2.0*G(mu_conj, a, mu, x_0))
    A = 2.0*(np.sinh(mu)+mu*np.cosh(mu)-a*x_0*np.sinh(mu*x_0))
    d = B/A
    return d.real

In [5]:
def d_phase_func(a, m, k1, k2, n1, n2, x_0):
    B = 3.0*m*(G(k1, a, m, x_0) + G(k2, a, m, x_0) + G(n1, a, m, x_0) + G(n2, a, m, x_0))
    A = 2.0*(np.sinh(m)+m*np.cosh(m)-a*x_0*np.sinh(m*x_0))
    d = B/A
    return d.real

In [6]:
w_c = [6.4771398, 10.199406, 12.171111, 13.218219, 13.540905, 13.127015, 11.714352, 7.9323942]
w_f = [6.4764458, 10.199064, 12.170777, 13.218143, 13.540667, 13.12674, 11.714092, 7.9320798]
a_c = [-6.3450342, -11.332399, -13.988385, -14.905533, -14.587681, -13.40822, -11.588376, -9.0473602]
a_f = [-6.3442514, -11.331898, -13.987908, -14.905437, -14.587432, -13.407994, -11.58822, -9.0472563]
x0 = np.linspace(0.35, 0.49, 8)
g = [5.091170205, 5.634962472, 6.49705811, 7.629131375, 9.002669738, 10.607968684, 12.467312024, 14.738425799]
i = 1.0j

In [7]:
cols = ['x0', 'delta', 'delta1', 'delta2', 'rho1', 'rho2', 'Stability', 'phi1', 'phi2', 'd11', 'd12', 'd21', 'd22']
df = pd.DataFrame(columns=cols)
df.head()

,x0,delta,delta1,delta2,rho1,rho2,Stability,phi1,phi2,d11,d12,d21,d22


In [8]:
for idx in range(len(x0)):
    mu_c = np.sqrt(-g[idx] + i*w_c[idx])
    mu_f = np.sqrt(-g[idx] + i*w_f[idx])
    kappa_c = mu_c + 2.0*mu_c.real
    nu_c = mu_c + 2.0*i*mu_c.imag
    kappa_f = mu_f + 2.0*mu_f.real
    nu_f = mu_f + 2.0*i*mu_f.imag
    kappa11 = mu_c + 2.0*mu_f.real
    kappa12 = mu_c - 2.0*mu_f.real
    nu11 = mu_c + 2.0*i*mu_f.imag
    nu12 = mu_c - 2.0*i*mu_f.imag
    kappa21 = mu_f + 2.0*mu_c.real
    kappa22 = mu_f - 2.0*mu_c.real
    nu21 = mu_f + 2.0*i*mu_c.imag
    nu22 = mu_f - 2.0*i*mu_c.imag
    phi1 = abs(phi_func(g[idx], a_c[idx], w_c[idx], x0[idx]))
    phi2 = abs(phi_func(g[idx], a_f[idx], w_f[idx], x0[idx]))
    d11 = d_func(mu_c, kappa_c, nu_c, a_c[idx], x0[idx])
    d12 = d_phase_func(a_c[idx], mu_c, kappa11, kappa12, nu11, nu12, x0[idx])
    d21 = d_phase_func(a_f[idx], mu_f, kappa21, kappa22, nu21, nu22, x0[idx])
    d22 = d_func(mu_f, kappa_f, nu_f, a_f[idx], x0[idx])
    delta = d11*d22 - d12*d21
    delta1 = -phi1*d22+phi2*d12
    delta2 = -phi2*d11+phi1*d21
    # bad solution
    if delta >= 0.0:
        delta1, delta2 = abs(delta1), abs(delta2)
        stab = 'Stable'
    else:
        delta1, delta2 = -abs(delta1), -abs(delta2)
        stab = 'Unstable'
    #
    rho1, rho2 = np.sqrt(delta1/delta), np.sqrt(delta2/delta)
    df.loc[idx] =[x0[idx], delta, delta1, delta2, rho1, rho2, stab, phi1, phi2, d11, d12, d21, d22]

In [9]:
df.head(10)

,x0,delta,delta1,delta2,rho1,rho2,Stability,phi1,phi2,d11,d12,d21,d22
0,0.35,2.994325e-08,3.690469e-08,3.691078e-08,1.110175,1.110266,Stable,0.000003,0.000003,-4.066398,-4.065724,-4.066379,-4.065705
1,0.37,2.143173e-08,5.449648e-07,5.450037e-07,5.042613,5.042793,Stable,0.000003,0.000003,-9.645265,-9.644576,-9.645192,-9.644503
2,0.39,2.801809e-08,3.344052e-07,3.344269e-07,3.454755,3.454867,Stable,0.000005,0.000005,-14.555276,-14.554332,-14.555162,-14.554218
3,0.41,1.248168e-09,1.200565e-06,1.200583e-06,31.013904,31.014125,Stable,0.000002,0.000002,-17.692811,-17.692559,-17.692790,-17.692538
4,0.43,4.183619e-09,7.310579e-07,7.310901e-07,13.219037,13.219328,Stable,0.000008,0.000008,-18.482446,-18.481632,-18.482450,-18.481636
5,0.45,-9.486826e-09,-7.134339e-07,-7.134704e-07,8.671943,8.672165,Unstable,0.000013,0.000013,-16.827038,-16.826178,-16.827161,-16.826301
6,0.47,-2.000775e-08,-1.302203e-07,-1.302268e-07,2.551175,2.551239,Unstable,0.000016,0.000016,-12.965394,-12.964754,-12.965607,-12.964967
7,0.49,-3.278697e-08,-2.041153e-08,-2.041269e-08,0.789018,0.789041,Unstable,0.000027,0.000027,-6.991332,-6.990893,-6.991585,-6.991146
